In [6]:
from sympy import symbols, Matrix, eye, simplify
from sympy import latex
from IPython.display import display, Math

# Define symbols
dt, k_re, mass, u = symbols('dt k_re mass u_in')

A21 = symbols('A21')  # Element of matrix A
B2 = symbols('B2')  # Element of matrix B
P11, P12, P21, P22 = symbols('P11 P12 P21 P22')  # Elements of covariance matrix P
P11_sim, P12_sim, P21_sim, P22_sim = symbols('P_pred11 P_pred12 P_pred21 P_pred22')  # Simulated covariance matrix elements
Q11, Q22 = symbols('Q11 Q22')  # Elements of process noise covariance matrix Q
R_mea = symbols('R_mea')  # Measurement noise covariance (scalar)

x_pred, vx_pred = symbols('x_pred vx_pred')  # Predicted state variables
y_mea, y_meaUD = symbols('y_mea y_meaUD')  # Measurement variable
x_step, vx_step = symbols('x_step vx_step')  # relative update
# System matrices
A = Matrix([[1, dt], 
            [A21, 1]])
B = Matrix([[0], [B2]])
C = Matrix([[1, 0]])
x_pred_vector = Matrix([[x_pred], [vx_pred]])

# Prediction step (before jumping)
x_temp = A * x_pred_vector + B * u  # Predicted state vector
x_temp[0] += x_step  
P_pred = A * Matrix([[P11, P12], [P21, P22]]) * A.T + Matrix([[Q11, 0], [0, Q22]])
jumping_step = Matrix([[x_step], [vx_step]])

# Measurement step(after jumping)
# Calculate Kalman gain
S = (C * P_pred * C.T)[0, 0] + R_mea  # Extract scalar from 1x1 matrix and add R
S_sim = symbols('S')  # Measurement prediction covariance
# K = P_pred * C.T / S  # 2x1 vector

# Joseph form update
x_temp_sim_11, x_temp_sim_12 = symbols('x_temp1 x_temp2')
x_temp_sim = Matrix([[x_temp_sim_11], [x_temp_sim_12]])
K1, K2 = symbols('K1 K2')  # Kalman gain components
# K = Matrix([[P11_sim/S_sim], [P21_sim/S_sim]])
K = Matrix([[K1], [K2]])
P_pred_sim = Matrix([[P11_sim, P12_sim], [P21_sim, P22_sim]])

I = eye(2)
IKC = I - K * C
P_update = IKC * P_pred_sim * IKC.T + K * R_mea * K.T

# compute update state vector
x_update = x_temp_sim + K * (y_meaUD - (C * (x_temp_sim))[0, 0])
x_update[1] += jumping_step[1]

# Simplify expressions
# Simplify all key expressions
x_temp_s = simplify(x_temp)
P_pred_s = simplify(P_pred)
S_s = simplify(S)
K_s = simplify(K)
P_update_s = simplify(P_update)
x_update_s = simplify(x_update)

# Display results in LaTeX
display(Math(r"x_{\text{temp}}: " + latex(x_temp_s)))
display(Math(r"P_{\text{pred}}: " + latex(P_pred_s)))
display(Math(r"S: " + latex(S_s)))
display(Math(r"K: " + latex(K_s)))
display(Math(r"P_{\text{update}}: " + latex(P_update_s)))
display(Math(r"x_{\text{update}}: " + latex(x_update_s)))
print("x_temp:", x_temp_s)
print("P_pred:", P_pred_s)
print("S:", S_s)
print("K:", K_s)
print("P_update:", P_update_s)
print("x_update:", x_update_s)






<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

x_temp: Matrix([[dt*vx_pred + x_pred + x_step], [A21*x_pred + B2*u_in + vx_pred]])
P_pred: Matrix([[P11 + P21*dt + Q11 + dt*(P12 + P22*dt), A21*(P11 + P21*dt) + P12 + P22*dt], [A21*P11 + P21 + dt*(A21*P12 + P22), A21*P12 + A21*(A21*P11 + P21) + P22 + Q22]])
S: P11 + P21*dt + Q11 + R_mea + dt*(P12 + P22*dt)
K: Matrix([[K1], [K2]])
P_update: Matrix([[K1**2*R_mea + P_pred11*(K1 - 1)**2, K1*K2*R_mea + K2*P_pred11*(K1 - 1) - P_pred12*(K1 - 1)], [K1*K2*R_mea + (K1 - 1)*(K2*P_pred11 - P_pred21), K2**2*R_mea - K2*P_pred12 + K2*(K2*P_pred11 - P_pred21) + P_pred22]])
x_update: Matrix([[-K1*(x_temp1 - y_meaUD) + x_temp1], [-K2*(x_temp1 - y_meaUD) + vx_step + x_temp2]])
